## Requirements

### Imports

In [1]:
import os
import torch
from pathlib import Path
from torchvision import models, transforms

In [2]:
from functools import partial
import os
import tempfile
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray import train
from ray.train import Checkpoint, get_checkpoint
from ray.tune.schedulers import ASHAScheduler
import ray.cloudpickle as pickle
from torch.utils.data import DataLoader, Dataset, Subset
from torch.optim.lr_scheduler import OneCycleLR, StepLR

g:\Benutzer\X_Ichigo_X\Dokumente\FH_Master\Ki_labor\git\mushroom_classification\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-12 20:34:11,293	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-01-12 20:34:11,903	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
import training

In [4]:
torch.manual_seed(42)

### Paths

In [5]:
SRC_DIR = Path.cwd()
ROOT_DIR = SRC_DIR.parent

tmp_folder = os.path.join(ROOT_DIR, 'ray_results')
DATA_DIR = os.path.join(ROOT_DIR, 'dataset')
PREPROCESSED_DIR = os.path.join(DATA_DIR, 'preprocessed')
CSV_PATH = os.path.join(DATA_DIR, 'csv_mappings', 'train.csv')

MODEL_DIR = os.path.join(ROOT_DIR, 'models')
BASELINE_DIR = os.path.join(MODEL_DIR, 'baselines')
RESULT_DIR = os.path.join(BASELINE_DIR, 'results')

# Set a custom temporary directory
os.environ["TMPDIR"] = os.path.join(tmp_folder, '.tmp')  # Replace "./temp_dir" with your desired path

### Load Data

In [6]:
BATCH_SIZE = 32

In [7]:
CLASS_NAMES = {
    0: "amanita",
    1: "boletus",
    2: "chantelle",
    3: "deterrimus",
    4: "rufus",
    5: "torminosus",
    6: "aurantiacum",
    7: "procera",
    8: "involutus",
    9: "russula"
}

### Enhanced ResNet

In [8]:
class EnhancedResNet(nn.Module):
    def __init__(self, num_classes=10, dropout_prob=0.5, l1=1024):
        super(EnhancedResNet, self).__init__()

        self.resnet = models.resnet50(pretrained=True)  
        self.resnet.fc = nn.Identity()
        self.dropout = nn.Dropout(p=dropout_prob)
        
        self.fc1 = nn.Linear(2048, l1)  
        self.fc2 = nn.Linear(l1, num_classes)
        
    def forward(self, x):
        x = self.resnet(x)  
        x = self.dropout(x) 
        x = torch.relu(self.fc1(x)) 
        x = self.fc2(x)  
        return x

### training

In [9]:
def train_model(config, data_dir=None):
    net = EnhancedResNet(dropout_prob = config["dropout"], l1 = config["l1"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)
    print(device)

    criterion = nn.CrossEntropyLoss()
    #optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)
    optimizer = optim.AdamW(net.parameters(), lr=config["lr"], weight_decay=1e-5)

    checkpoint = get_checkpoint()
    if checkpoint:
        with checkpoint.as_directory() as checkpoint_dir:
            data_path = Path(checkpoint_dir) / "data.pkl"
            with open(data_path, "rb") as fp:
                checkpoint_state = pickle.load(fp)
            start_epoch = checkpoint_state["epoch"]
            net.load_state_dict(checkpoint_state["net_state_dict"])
            optimizer.load_state_dict(checkpoint_state["optimizer_state_dict"])
    else:
        start_epoch = 0


    trainloader, valloader, test_abs = training.get_data_loaders(PREPROCESSED_DIR, CSV_PATH, config["batch_size"])

    for epoch in range(start_epoch, 10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print(
                    "[%d, %5d] loss: %.3f"
                    % (epoch + 1, i + 1, running_loss / epoch_steps)
                )
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        checkpoint_data = {
            "epoch": epoch,
            "net_state_dict": net.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
        }

        with tempfile.TemporaryDirectory() as checkpoint_dir:
            data_path = Path(checkpoint_dir) / "data.pkl"
            with open(data_path, "wb") as fp:
                pickle.dump(checkpoint_data, fp)

            checkpoint = Checkpoint.from_directory(checkpoint_dir)
            train.report(
                {"loss": val_loss / val_steps, "accuracy": correct / total},
                checkpoint=checkpoint,
            )

    print("Finished Training")

### Test set accracy

In [10]:
def test_accuracy(net, device="cpu"):
    
    trainloader, valloader, testloader = training.get_data_loaders(PREPROCESSED_DIR, CSV_PATH, BATCH_SIZE)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

### Config

In [11]:
config = {
    "l1": tune.choice([512,1024,2048,4096]),
    "lr": tune.loguniform(1e-5, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16, 32, 64]),
    "dropout" : tune.loguniform(0.5, 1),
}

### tune

In [12]:
def start_tune (num_samples=20, max_num_epochs=20, gpus_per_trial=1):
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2,
)
    #optimizer = optim.AdamW(EnhancedResNet.parameters(), lr=config["lr"], weight_decay=1e-5)
    #scheduler = StepLR(optimizer, step_size=1, gamma=0.5)

    result = tune.run(
        partial(train_model),
        resources_per_trial={"cpu": 8, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=None,
        storage_path=tmp_folder
    )

    best_trial = result.get_best_trial("loss", "min", "last")
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final validation loss: {best_trial.last_result['loss']}")
    print(f"Best trial final validation accuracy: {best_trial.last_result['accuracy']}")

    best_trained_model = EnhancedResNet(best_trial.config["l1"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint = result.get_best_checkpoint(trial=best_trial, metric="accuracy", mode="max")
    with best_checkpoint.as_directory() as checkpoint_dir:
        data_path = Path(checkpoint_dir) / "data.pkl"
        with open(data_path, "rb") as fp:
            best_checkpoint_data = pickle.load(fp)

        best_trained_model.load_state_dict(best_checkpoint_data["net_state_dict"])
        test_acc = test_accuracy(best_trained_model, device)
        print("Best trial test set accuracy: {}".format(test_acc))
    return result

In [13]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"

print(device)


cuda:0


In [ ]:

result = start_tune(num_samples=20)

2025-01-12 20:34:19,073	INFO worker.py:1821 -- Started a local Ray instance.
2025-01-12 20:34:23,998	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2025-01-12 20:34:24,000	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(func pid=47024) g:\Benutzer\X_Ichigo_X\Dokumente\FH_Master\Ki_labor\git\mushroom_classification\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(func pid=47024)   warnings.warn(
(func pid=47024) g:\Benutzer\X_Ichigo_X\Dokumente\FH_Master\Ki_labor\git\mushroom_classification\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
(func pid=47024)   warnings.warn(msg)
(func pid=47024) g:\Benutzer\X_Ichigo_X\Dokumente\FH_Master\Ki_labor\git\mushroom_classification\src\training.py:69: FutureWarning: You are using `torch.load` with `weights_only=False

(func pid=47024) cuda:0


Trial name,accuracy,loss,should_checkpoint
train_model_3a5ba_00000,0.0873239,2.30742,True
train_model_3a5ba_00001,0.794366,0.654514,True
train_model_3a5ba_00002,0.0985915,2.30845,True
train_model_3a5ba_00003,0.8,0.781084,True
train_model_3a5ba_00004,0.0957746,2.35852,True
train_model_3a5ba_00005,0.11831,2.2988,True
train_model_3a5ba_00006,0.833803,0.672984,True
train_model_3a5ba_00007,0.422535,1.69277,True
train_model_3a5ba_00008,0.104225,2.98806,True
train_model_3a5ba_00009,0.622535,1.51813,True


(func pid=47024) Checkpoint successfully created at: Checkpoint(filesystem=local, path=g:/Benutzer/X_Ichigo_X/Dokumente/FH_Master/Ki_labor/git/mushroom_classification/ray_results/train_model_2025-01-12_20-34-24/train_model_3a5ba_00000_0_batch_size=64,dropout=0.5889,l1=512,lr=0.0628_2025-01-12_20-34-24/checkpoint_000000)
(func pid=47024) Checkpoint successfully created at: Checkpoint(filesystem=local, path=g:/Benutzer/X_Ichigo_X/Dokumente/FH_Master/Ki_labor/git/mushroom_classification/ray_results/train_model_2025-01-12_20-34-24/train_model_3a5ba_00000_0_batch_size=64,dropout=0.5889,l1=512,lr=0.0628_2025-01-12_20-34-24/checkpoint_000001)
(func pid=47024) Checkpoint successfully created at: Checkpoint(filesystem=local, path=g:/Benutzer/X_Ichigo_X/Dokumente/FH_Master/Ki_labor/git/mushroom_classification/ray_results/train_model_2025-01-12_20-34-24/train_model_3a5ba_00000_0_batch_size=64,dropout=0.5889,l1=512,lr=0.0628_2025-01-12_20-34-24/checkpoint_000002)
(func pid=47024) Checkpoint succes

...

(func pid=47972) Checkpoint successfully created at: Checkpoint(filesystem=local, path=g:/Benutzer/X_Ichigo_X/Dokumente/FH_Master/Ki_labor/git/mushroom_classification/ray_results/train_model_2025-01-12_20-34-24/train_model_3a5ba_00012_12_batch_size=64,dropout=0.7493,l1=1024,lr=0.0085_2025-01-12_20-34-24/checkpoint_000000)
(func pid=47972) Checkpoint successfully created at: Checkpoint(filesystem=local, path=g:/Benutzer/X_Ichigo_X/Dokumente/FH_Master/Ki_labor/git/mushroom_classification/ray_results/train_model_2025-01-12_20-34-24/train_model_3a5ba_00012_12_batch_size=64,dropout=0.7493,l1=1024,lr=0.0085_2025-01-12_20-34-24/checkpoint_000001)
(func pid=47972) Checkpoint successfully created at: Checkpoint(filesystem=local, path=g:/Benutzer/X_Ichigo_X/Dokumente/FH_Master/Ki_labor/git/mushroom_classification/ray_results/train_model_2025-01-12_20-34-24/train_model_3a5ba_00012_12_batch_size=64,dropout=0.7493,l1=1024,lr=0.0085_2025-01-12_20-34-24/checkpoint_000002)
(func pid=47972) Checkpoint 

(func pid=47972) Finished Training


(func pid=27604) g:\Benutzer\X_Ichigo_X\Dokumente\FH_Master\Ki_labor\git\mushroom_classification\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(func pid=27604)   warnings.warn(
(func pid=27604) g:\Benutzer\X_Ichigo_X\Dokumente\FH_Master\Ki_labor\git\mushroom_classification\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
(func pid=27604)   warnings.warn(msg)
(func pid=27604) g:\Benutzer\X_Ichigo_X\Dokumente\FH_Master\Ki_labor\git\mushroom_classification\src\training.py:69: FutureWarning: You are using `torch.load` with `weights_only=False

(func pid=27604) cuda:0


(func pid=27604) Checkpoint successfully created at: Checkpoint(filesystem=local, path=g:/Benutzer/X_Ichigo_X/Dokumente/FH_Master/Ki_labor/git/mushroom_classification/ray_results/train_model_2025-01-12_20-34-24/train_model_3a5ba_00013_13_batch_size=16,dropout=0.8129,l1=4096,lr=0.0039_2025-01-12_20-34-24/checkpoint_000000)
(func pid=27604) Checkpoint successfully created at: Checkpoint(filesystem=local, path=g:/Benutzer/X_Ichigo_X/Dokumente/FH_Master/Ki_labor/git/mushroom_classification/ray_results/train_model_2025-01-12_20-34-24/train_model_3a5ba_00013_13_batch_size=16,dropout=0.8129,l1=4096,lr=0.0039_2025-01-12_20-34-24/checkpoint_000001)
(func pid=27604) Checkpoint successfully created at: Checkpoint(filesystem=local, path=g:/Benutzer/X_Ichigo_X/Dokumente/FH_Master/Ki_labor/git/mushroom_classification/ray_results/train_model_2025-01-12_20-34-24/train_model_3a5ba_00013_13_batch_size=16,dropout=0.8129,l1=4096,lr=0.0039_2025-01-12_20-34-24/checkpoint_000002)


In [ ]:
best_trial = result.get_best_trial("loss", "min", "last")
print(f"Best trial config: {best_trial.config}")
print(f"Best trial final validation loss: {best_trial.last_result['loss']}")
print(f"Best trial final validation accuracy: {best_trial.last_result['accuracy']}")